In [1]:
import requests as req
import pandas as pd

season = '20212022'

#GET ALL THE SKATERS FOR THE SELECTED SEASON
df = req.get(f'https://api.nhle.com/stats/rest/en/skater/bios?limit=-1&start=0&cayenneExp=seasonId={season}')
df = df.json()
Skaters = pd.json_normalize(df, "data")

#RENAME COLUMN 'skaterFullName' to 'Name AND SELECT/ORDER COLUMNS
Skaters = Skaters.rename(columns={"skaterFullName":"Name"})
Skaters = Skaters[['birthDate','draftOverall','draftYear','height','nationalityCode','playerId','shootsCatches','weight','positionCode','Name']]

#GET ALL THE GOALIES FOR THE SELECTED SEASON
df = req.get(f'https://api.nhle.com/stats/rest/en/goalie/bios?limit=-1&start=0&cayenneExp=seasonId={season}')
df = df.json()
Goalies = pd.json_normalize(df, "data")

#RENAME COLUMN 'goalieFullName' to 'Name, ADD COLUMN: 'positionCode' AND SELECT/ORDER COLUMNS
Goalies = Goalies.rename(columns={"goalieFullName":"Name"})
Goalies['positionCode'] = 'G'
Goalies = Goalies[['birthDate','draftOverall','draftYear','height','nationalityCode','playerId','shootsCatches','weight','positionCode','Name']]

#COMBINE THE TWO DATAFRAMES: 'Skaters' AND 'Goalies' - ADD THE THE COLUMN: 'Season'
Players = pd.concat([Skaters,Goalies])
Players['Season'] = season

#RE-OREDER THE COLUMNS TO MATCH THE SQL SCRIPT AND CONVERT THE DATATYPES TO AVOID FLOATS
Players = Players[['Season','birthDate','draftOverall','draftYear','height','nationalityCode','playerId','shootsCatches','weight','positionCode','Name']]
Players = Players.convert_dtypes()

#DROP DUPLICATE PLAYERS
Players = Players.drop_duplicates(subset=['playerId'])

#GET ALL THE TEAMS FOR ALL SEASONS AND CONVERT THE DATATYPES TO AVOID FLOATS
df = req.get(f'https://api.nhle.com/stats/rest/en/team')
df = df.json()
Teams = pd.json_normalize(df, "data")
Teams = Teams.convert_dtypes()

#CREATE A LIST FOR SCHEDULE
schedule = []

#LOOP THROUGH EACH TEAM TO GET THE SCHEDULE FOR EVERY TEAM
for triCode in Teams['triCode']:
    url = f'https://api-web.nhle.com/v1/club-schedule-season/{triCode}/{season}'
    response = req.get(url)
    data = response.json()
    data = pd.json_normalize(data, "games")
    schedule.append(data)
    
schedule = pd.concat(schedule, ignore_index=True)
schedule = schedule.drop_duplicates(subset=['id'])

#SELECT/ORDER COLUMNS AND CONVERT THE DATATYPES TO AVOID FLOATS
schedule = schedule[['id','season','gameType','gameDate','startTimeUTC','gameState','awayTeam.abbrev','awayTeam.score',
                     'homeTeam.abbrev','homeTeam.score','gameOutcome.lastPeriodType']]
schedule = schedule.convert_dtypes()

#ONLY INCLUDE REGULAR AND PLAYOFF GAMES AND SORT THE DATA
no_preseason = schedule['gameType'].isin([2,3])
schedule = schedule[no_preseason]
schedule = schedule.sort_values(by=['id'])

#CREATE A LIST FOR PBP
pbp = []

#LOOP THROUGH EACH GAME TO GET THE PLAY-BY-PLAY FOR THE FULL SEASON
for Game in schedule['id']:
    url = f'https://api-web.nhle.com/v1/gamecenter/{Game}/play-by-play'
    response = req.get(url)
    data = response.json()
    data = pd.json_normalize(data, "plays")
    data['GameID'] = Game
    pbp.append(data)
    
pbp = pd.concat(pbp, ignore_index=True)

#ADD COLUMNS THAT MIGHT NOT EXISTS IN THE OLDER SEASONS
pbp['LINK_PBP'] = pbp.get('LINK_PBP','')
pbp['situationCode'] = pbp.get('situationCode','')
pbp['homeTeamDefendingSide'] = pbp.get('homeTeamDefendingSide','')
pbp['details.losingPlayerId'] = pbp.get('details.losingPlayerId','')
pbp['details.winningPlayerId'] = pbp.get('details.winningPlayerId','')
pbp['details.xCoord'] = pbp.get('details.xCoord','')
pbp['details.yCoord'] = pbp.get('details.yCoord','')
pbp['details.zoneCode'] = pbp.get('details.zoneCode','')
pbp['details.reason'] = pbp.get('details.reason','')
pbp['details.hittingPlayerId'] = pbp.get('details.hittingPlayerId','')
pbp['details.hitteePlayerId'] = pbp.get('details.hitteePlayerId','')
pbp['details.playerId'] = pbp.get('details.playerId','')
pbp['details.shotType'] = pbp.get('details.shotType','')
pbp['details.shootingPlayerId'] = pbp.get('details.shootingPlayerId','')
pbp['details.awaySOG'] = pbp.get('details.awaySOG','')
pbp['details.homeSOG'] = pbp.get('details.homeSOG','')
pbp['details.blockingPlayerId'] = pbp.get('details.blockingPlayerId','')
pbp['details.assist2PlayerId'] = pbp.get('details.assist2PlayerId','')
pbp['details.assist2PlayerTotal'] = pbp.get('details.assist2PlayerTotal','')
pbp['details.secondaryReason'] = pbp.get('details.secondaryReason','')
pbp['details.typeCode'] = pbp.get('details.typeCode','')
pbp['details.drawnByPlayerId'] = pbp.get('details.drawnByPlayerId','')
pbp['details.servedByPlayerId'] = pbp.get('details.servedByPlayerId','')

#SELECT/ORDER COLUMNS AND CONVERT THE DATATYPES TO AVOID FLOATS
pbp = pbp[['GameID','LINK_PBP','eventId','periodDescriptor.number','timeInPeriod','situationCode','homeTeamDefendingSide','typeCode',
           'typeDescKey','sortOrder','details.eventOwnerTeamId','details.losingPlayerId','details.winningPlayerId','details.xCoord','details.yCoord',
           'details.zoneCode','details.reason','details.hittingPlayerId','details.hitteePlayerId','details.playerId','details.shotType',
           'details.shootingPlayerId','details.goalieInNetId','details.awaySOG','details.homeSOG','details.blockingPlayerId',
           'details.scoringPlayerId','details.scoringPlayerTotal','details.assist1PlayerId','details.assist1PlayerTotal',
           'details.assist2PlayerId','details.assist2PlayerTotal','details.awayScore','details.homeScore','details.secondaryReason',
           'details.typeCode','details.descKey','details.duration','details.committedByPlayerId','details.drawnByPlayerId','details.servedByPlayerId']]
pbp = pbp.convert_dtypes()

#CREATE A LIST FOR shifts
shifts = []

#LOOP THROUGH EACH GAME TO GET THE shifts FOR THE FULL SEASON
for Game in schedule['id']:
    url = f'https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId={Game}'
    response = req.get(url)
    data = response.json()
    data = pd.json_normalize(data, "data")
    shifts.append(data)

shifts = pd.concat(shifts, ignore_index=True)

#ADD COLUMNS THAT MIGHT NOT EXISTS IN THE OLDER SEASONS
shifts['gameId'] = shifts.get('gameId','')
shifts['endTime'] = shifts.get('endTime','')
shifts['period'] = shifts.get('period','')
shifts['playerId'] = shifts.get('playerId','')
shifts['shiftNumber'] = shifts.get('shiftNumber','')
shifts['startTime'] = shifts.get('startTime','')
shifts['teamAbbrev'] = shifts.get('teamAbbrev','')

#SELECT/ORDER COLUMNS AND CONVERT THE DATATYPES TO AVOID FLOATS
shifts = shifts[['gameId','endTime','period','playerId','shiftNumber','startTime','teamAbbrev']]
shifts = shifts.convert_dtypes()

#WRITE THE DATA TO CSV FILES 
Players.to_csv(r'C:\Public\NHL\Players.csv',index=False, header=True)
Teams.to_csv(r'C:\Public\NHL\Teams.csv',index=False, header=True)
schedule.to_csv(r'C:\Public\NHL\Schedule.csv',index=False, header=True)
pbp.to_csv(r'C:\Public\NHL\pbp.csv',index=False, header=True)
shifts.to_csv(r'C:\Public\NHL\shift.csv',index=False, header=True)